In [2]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
df = pd.read_csv('df_preproces0414.csv')
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,Time_hour,Time_min
0,61860.0,0.530985,-1.368655,0.893127,0.370692,-1.301300,0.413628,-0.393875,0.218721,0.812112,...,-0.148537,0.340996,-0.085972,0.907955,-0.074088,0.056354,5.736572,0,17.0,1031.000000
1,164437.0,-7.357404,-1.601556,-4.676849,1.507472,-3.048604,1.264281,2.518449,0.905625,-0.676205,...,-1.438854,0.313489,0.776204,-0.102063,-0.397369,-1.945864,6.152009,0,45.0,2740.616667
2,78471.0,-0.709215,-0.402005,2.300180,-2.461564,-0.759022,-0.127262,-0.333105,0.132349,-2.577745,...,0.044342,0.045298,-0.004456,-0.458079,0.086695,0.072301,3.414443,0,21.0,1307.850000
3,172386.0,-1.100930,1.708435,-0.992206,-0.824640,0.499438,-0.617786,0.604869,0.357170,0.580096,...,0.185649,0.558143,-0.308057,0.088361,0.344577,0.052029,2.297573,0,47.0,2873.100000
4,39374.0,-1.084659,1.875061,0.350536,3.167242,-0.391817,0.627411,-0.557509,1.254159,-1.153587,...,0.009724,-0.064580,-0.225990,0.279869,0.234683,0.045528,2.684440,0,10.0,656.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213600,73826.0,-0.297228,0.837941,-0.299616,-1.110196,2.567544,3.253144,0.098229,0.937040,-0.501688,...,-0.050210,0.935690,-0.006366,0.088117,0.247066,0.091383,0.636577,0,20.0,1230.433333
213601,55773.0,-0.608749,0.659941,1.560450,1.452568,0.088723,-0.349799,0.301728,0.220852,0.005055,...,-0.047337,0.357099,-0.191730,-0.236428,0.409219,0.208162,1.899118,0,15.0,929.550000
213602,159100.0,2.041114,-0.153033,-1.190310,0.231333,0.016572,-0.720266,0.034818,-0.133703,0.393756,...,0.295240,-0.352259,-0.306028,0.200266,-0.078892,-0.074601,1.091923,0,44.0,2651.666667
213603,78075.0,-0.568765,1.135885,1.193505,0.375710,0.483989,0.259682,0.070903,-0.615915,-0.872376,...,-0.017092,-0.339691,-0.662538,0.202004,0.189188,0.148592,0.828552,0,21.0,1301.250000


In [3]:
X = df.drop('Class',axis=1)
Y = df['Class']

In [4]:
from sklearn.model_selection import train_test_split

ran = 42
train_x,test_x, train_y,test_y = train_test_split(X,Y, random_state=ran, stratify=df['Class'])

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class TimeFraudRateEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, bins=100):
        self.bins = bins
        self.bin_edges_ = None
        self.fraud_rate_dic_ = None

    def fit(self, X, y):
        X_ = X.copy()
        X_['Time_min'] = X_['Time'] / 60

        time_bins_series, bin_edges = pd.cut(X_['Time_min'], bins=self.bins, retbins=True)
        fraud_rate_dic = pd.Series(y).groupby(time_bins_series).mean().to_dict()

        self.bin_edges_ = bin_edges
        self.fraud_rate_dic_ = fraud_rate_dic

        return self

    def transform(self, X):
        X = X.copy()
        X['Time_min'] = X['Time'] / 60

        # Binning
        X['Time_bin'] = pd.cut(X['Time_min'], bins=self.bin_edges_, include_lowest=True)

        # Time_bin 을 Label Encoding (숫자형 변환)
        X['Time_bin_encoded'] = pd.cut(X['Time_min'], bins=self.bin_edges_, include_lowest=True).cat.codes

        # Fraud rate 매핑
        X['Time_fraud_rate'] = X['Time_bin'].map(self.fraud_rate_dic_).fillna(0)

        # 중간 column 정리 (원하시면 Time_min 은 제거)
        X = X.drop(columns=['Time_min'])

        return X


In [6]:
encoder = TimeFraudRateEncoder(bins=100)
train_transformed = encoder.fit_transform(train_x, train_y)
test_transformed = encoder.transform(test_x)

In [7]:
del train_transformed['Time_bin']
del test_transformed['Time_bin']

In [8]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(random_state=ran)
x_smotomek, y_smotomek = smote_tomek.fit_resample(train_transformed, train_y)

#smote = SMOTE(random_state=ran)
#x_smote, y_smote = smote.fit_resample(train_transformed, train_y)

oss = OneSidedSelection(random_state=ran)
x_oss, y_oss = oss.fit_resample(train_transformed, train_y)

In [9]:
y_smotomek.value_counts(), y_oss.value_counts() # y_smote.value_counts()

(Class
 0    159654
 1    159654
 Name: count, dtype: int64,
 Class
 0    159380
 1       265
 Name: count, dtype: int64)

In [13]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import all_estimators
from sklearn.metrics import f1_score
import pandas as pd

# sklearn의 모든 classifier 가져오기
all_classifiers = all_estimators(type_filter='classifier')

# 결과 저장용 리스트
results = []

# 사용할 데이터셋
X_train, y_train = x_oss, y_oss 
X_test, y_test = test_transformed, test_y

# 모든 classifier 반복
for name, ClassifierClass in all_classifiers:
    try:
        # 인스턴스 생성 (필요 시 기본 파라미터 사용)
        model = ClassifierClass()

        # 모델 학습
        model.fit(X_train, y_train)

        # 예측
        preds = model.predict(X_test)

        # 평가 (F1 score)
        f1 = f1_score(y_test, preds, average='macro')

        # 결과 저장
        results.append({'Model': name, 'F1 Score': f1})
        print(f"{name}: F1 Score = {f1:.4f}")

    except Exception as e:
        # 에러나는 모델은 스킵
        print(f"{name}: Error")
        continue

AdaBoostClassifier: F1 Score = 0.8854
BaggingClassifier: F1 Score = 0.9249
BernoulliNB: F1 Score = 0.8773
CalibratedClassifierCV: F1 Score = 0.7822
CategoricalNB: Error
ClassifierChain: Error
ComplementNB: Error
DecisionTreeClassifier: F1 Score = 0.8762
DummyClassifier: F1 Score = 0.4996
ExtraTreeClassifier: F1 Score = 0.8777
ExtraTreesClassifier: F1 Score = 0.9366
FixedThresholdClassifier: Error
GaussianNB: F1 Score = 0.6325
GaussianProcessClassifier: Error
GradientBoostingClassifier: F1 Score = 0.7653
HistGradientBoostingClassifier: F1 Score = 0.8219
KNeighborsClassifier: F1 Score = 0.6196
LabelPropagation: Error
LabelSpreading: Error
LinearDiscriminantAnalysis: F1 Score = 0.9171
LinearSVC: F1 Score = 0.8622
LogisticRegression: F1 Score = 0.8764
LogisticRegressionCV: F1 Score = 0.8920
MLPClassifier: F1 Score = 0.5602
MultiOutputClassifier: Error
MultinomialNB: Error
NearestCentroid: F1 Score = 0.3257
NuSVC: Error
OneVsOneClassifier: Error
OneVsRestClassifier: Error
OutputCodeClassifi

In [14]:
# 결과 DataFrame
results_df = pd.DataFrame(results)

# 성능 순으로 정렬
results_df = results_df.sort_values(by='F1 Score', ascending=False).reset_index(drop=True)

# 출력
print("\n=== Model Ranking (by F1 Score) ===")
results_df


=== Model Ranking (by F1 Score) ===


,Model,F1 Score
0,ExtraTreesClassifier,0.936615
1,RandomForestClassifier,0.933864
2,BaggingClassifier,0.924887
3,LinearDiscriminantAnalysis,0.917051
4,LogisticRegressionCV,0.892002
5,AdaBoostClassifier,0.885364
6,ExtraTreeClassifier,0.877710
7,BernoulliNB,0.877310
8,LogisticRegression,0.876445
9,DecisionTreeClassifier,0.876198


In [15]:
# Top 3 모델명 및 f1 score 기반 weight
top3_models = results_df.head(3)['Model'].tolist()
weights = results_df.head(3)['F1 Score'].tolist()

print("\nTop 3 Models:", top3_models)
print("Weights:", weights)


Top 3 Models: ['ExtraTreesClassifier', 'RandomForestClassifier', 'BaggingClassifier']
Weights: [0.9366150925919112, 0.9338638066517639, 0.9248874760886688]


In [17]:
from sklearn.ensemble import VotingClassifier
# 전체 classifier dictionary
all_classifiers_dict = {name: ClassifierClass for name, ClassifierClass in all_classifiers}

# Top 3 모델 인스턴스 생성
top3_instances = [(name, all_classifiers_dict[name]()) for name in top3_models]

# VotingClassifier 앙상블 (soft voting + 성능 기반 weight)
ensemble_model = VotingClassifier(estimators=top3_instances, voting='soft', weights=weights)
ensemble_model.fit(X_train, y_train)

# 예측 및 평가
ensemble_preds = ensemble_model.predict(X_test)
print("\n=== Ensemble Model Classification Report ===")
print(classification_report(y_test, ensemble_preds))


=== Ensemble Model Classification Report ===


NameError: name 'classification_report' is not defined

In [18]:
from sklearn.metrics import classification_report # oss

ensemble_preds = ensemble_model.predict(X_test)
print("\n=== Ensemble Model Classification Report ===")
print(classification_report(y_test, ensemble_preds))


=== Ensemble Model Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     53314
           1       0.97      0.78      0.87        88

    accuracy                           1.00     53402
   macro avg       0.99      0.89      0.93     53402
weighted avg       1.00      1.00      1.00     53402



In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import all_estimators
from sklearn.metrics import f1_score
import pandas as pd

# sklearn의 모든 classifier 가져오기
all_classifiers = all_estimators(type_filter='classifier')

# 결과 저장용 리스트
results = []

# 사용할 데이터셋
X_train, y_train = X_smotomek, y_smotomek 
X_test, y_test = test_transformed, test_y

# 모든 classifier 반복
for name, ClassifierClass in all_classifiers:
    try:
        # 인스턴스 생성 (필요 시 기본 파라미터 사용)
        model = ClassifierClass()

        # 모델 학습
        model.fit(X_train, y_train)

        # 예측
        preds = model.predict(X_test)

        # 평가 (F1 score)
        f1 = f1_score(y_test, preds, average='macro')

        # 결과 저장
        results.append({'Model': name, 'F1 Score': f1})
        print(f"{name}: F1 Score = {f1:.4f}")

    except Exception as e:
        # 에러나는 모델은 스킵
        print(f"{name}: Error")
        continue

AdaBoostClassifier: F1 Score = 0.5924
BaggingClassifier: F1 Score = 0.8676
BernoulliNB: F1 Score = 0.7494
CalibratedClassifierCV: F1 Score = 0.5590
CategoricalNB: Error
ClassifierChain: Error
ComplementNB: Error
DecisionTreeClassifier: F1 Score = 0.7536
DummyClassifier: F1 Score = 0.4996
ExtraTreeClassifier: F1 Score = 0.7638
ExtraTreesClassifier: F1 Score = 0.9336
FixedThresholdClassifier: Error
GaussianNB: F1 Score = 0.6163
GaussianProcessClassifier: Error
GradientBoostingClassifier: F1 Score = 0.7180
HistGradientBoostingClassifier: F1 Score = 0.8965
KNeighborsClassifier: F1 Score = 0.5613
LabelPropagation: Error
LabelSpreading: Error
LinearDiscriminantAnalysis: F1 Score = 0.6077
LinearSVC: F1 Score = 0.5761
LogisticRegression: F1 Score = 0.5749
LogisticRegressionCV: F1 Score = 0.6502
MLPClassifier: F1 Score = 0.7476
MultiOutputClassifier: Error
MultinomialNB: Error
NearestCentroid: F1 Score = 0.3258


In [ ]:
from sklearn.ensemble import VotingClassifier
# Top 3 모델명 및 f1 score 기반 weight
top3_models = results_df.head(3)['Model'].tolist()
weights = results_df.head(3)['F1 Score'].tolist()

print("\nTop 3 Models:", top3_models)
print("Weights:", weights)

# 전체 classifier dictionary
all_classifiers_dict = {name: ClassifierClass for name, ClassifierClass in all_classifiers}

# Top 3 모델 인스턴스 생성
top3_instances = [(name, all_classifiers_dict[name]()) for name in top3_models]

# VotingClassifier 앙상블 (soft voting + 성능 기반 weight)
ensemble_model = VotingClassifier(estimators=top3_instances, voting='soft', weights=weights)
ensemble_model.fit(X_train, y_train)

# 예측 및 평가
ensemble_preds = ensemble_model.predict(X_test)
print("\n=== Ensemble Model Classification Report ===")
print(classification_report(y_test, ensemble_preds))

In [ ]:
from sklearn.metrics import classification_report

ensemble_preds = ensemble_model.predict(X_test)
print("\n=== Ensemble Model Classification Report ===")
print(classification_report(y_test, ensemble_preds))

### 데이터에 따라 앙상블

In [12]:
from sklearn.utils import all_estimators

# 분류기만 가져오기
all_classifiers = all_estimators(type_filter='classifier')

# 딕셔너리화
all_classifiers_dict = {name: ClassifierClass for name, ClassifierClass in all_classifiers}

# XGBoost 수동으로 추가
from xgboost import XGBClassifier
all_classifiers_dict['XGBoost'] = XGBClassifier


In [15]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score

print("SMOTETomek score 기반 weight 계산...")

# 모델 별 f1 score 저장
model_scores = []

for name, model_class in smote_top3_instances:
    model = model_class
    model.fit(x_smotomek, y_smotomek)
    preds = model.predict(test_transformed)
    score = f1_score(test_y, preds)
    model_scores.append(score)
    print(f"{name} f1 score: {score:.4f}")

# 가중치 비율 계산 (전체 합 1 로 정규화)
total_score = sum(model_scores)
smote_weights = [score / total_score for score in model_scores]

print(f"SMOTETomek Weights (by score): {smote_weights}")

# VotingClassifier with computed weights
smote_ensemble = VotingClassifier(
    estimators=smote_top3_instances,
    voting='soft',
    weights=smote_weights
)
smote_ensemble.fit(x_smotomek, y_smotomek)
smote_probs = smote_ensemble.predict_proba(test_transformed)[:, 1]


SMOTETomek score 기반 weight 계산...
ExtraTreesClassifier f1 score: 0.8780
RandomForestClassifier f1 score: 0.8500
BaggingClassifier f1 score: 0.6923
SMOTETomek Weights (by score): [0.3627766365644743, 0.351187938451998, 0.2860354249835278]


In [31]:
smote_ensemble = VotingClassifier(
    estimators=smote_top3_instances,
    voting='soft',
    weights=[0.65, 0.35,0]  # 예: ExtraTrees 0.7, RandomForest 0.3
)
smote_ensemble.fit(x_smotomek, y_smotomek)
smote_probs = smote_ensemble.predict_proba(test_transformed)[:, 1]


In [46]:
from sklearn.metrics import classification_report

# threshold 적용
smote_preds = (smote_probs >= 0.65).astype(int)

# classification report 출력
print(classification_report(test_y, smote_preds))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     53314
           1       0.96      0.80      0.87        88

    accuracy                           1.00     53402
   macro avg       0.98      0.90      0.93     53402
weighted avg       1.00      1.00      1.00     53402



In [33]:
result_df = pd.read_csv('Result.csv')
result_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,163828,1.803825,-0.515611,-1.849958,0.388327,-0.109106,-0.756433,-0.071282,0.046801,1.226484,...,-0.186078,-0.659247,0.090988,-0.587207,-0.201435,-0.082970,-0.045304,-0.024865,106.16,NaN
1,161280,-0.547158,1.851470,-3.142172,-0.434762,1.905043,-0.955582,0.762074,0.533940,-1.190402,...,0.167646,0.338157,-0.277606,-0.287396,-0.269441,0.529420,-0.267365,-0.025603,0.76,NaN
2,151942,-0.326046,-0.217706,0.529752,-0.210276,0.381230,0.845321,-0.012136,0.122161,-1.551175,...,0.160597,0.627550,0.059524,-0.986212,-0.561194,0.203248,0.084215,0.197627,62.00,NaN
3,78663,-1.192746,0.589859,1.437212,-1.465846,0.904997,0.961699,0.720329,-0.107361,0.201824,...,-0.067400,0.048398,-0.502046,-1.224348,0.502369,1.053619,-0.751317,-0.139799,20.00,NaN
4,116781,1.348242,-1.475166,-1.262404,0.105946,0.281289,1.895419,-0.398966,0.538657,0.897916,...,-0.184420,-0.989198,0.261410,-0.963344,-0.880669,0.197854,-0.058803,-0.019419,288.93,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71197,66038,-0.882345,1.234006,1.835280,-0.543689,0.421677,-0.947680,1.170509,-0.349857,-0.392525,...,-0.316629,-0.752300,-0.234014,0.290883,0.275506,-0.012386,-0.058481,-0.085526,8.99,NaN
71198,125077,2.048996,-0.366883,-2.543852,-0.728626,2.387568,3.318339,-0.477944,0.791325,0.403268,...,-0.292818,-0.835277,0.380947,0.698226,-0.265129,0.219179,-0.052301,-0.066783,1.79,NaN
71199,154041,2.219642,-0.691482,-1.586185,-0.597533,-0.308073,-0.761185,-0.362941,-0.233413,-0.437414,...,-0.190701,-0.063660,0.139674,0.469916,-0.038320,0.775403,-0.078470,-0.063665,9.00,NaN
71200,40573,1.236821,0.055497,0.617927,0.882157,-0.590356,-0.588493,-0.109363,-0.124856,0.554266,...,-0.087457,0.030614,-0.088960,0.465463,0.603302,0.437325,-0.012067,0.010836,1.00,NaN


In [35]:
result_df['Time_min'] = result_df['Time'] / 60
result_df = encoder.transform(result_df)
result_df['Amount'] = np.log1p(result_df['Amount'])
result_df = encoder.transform(result_df)

In [36]:
result_df['Time_hour'] = result_df['Time'] // 3600

In [37]:
result_df = encoder.transform(result_df)

In [40]:
x_smotomek.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Time_hour', 'Time_bin_encoded', 'Time_fraud_rate'],
      dtype='object')

In [52]:
feature_columns = x_smotomek.columns  # 이전 train_x feature 컬럼 사용
result_df = result_df[feature_columns]
result_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Time_hour,Time_bin_encoded,Time_fraud_rate
0,163828,1.803825,-0.515611,-1.849958,0.388327,-0.109106,-0.756433,-0.071282,0.046801,1.226484,...,0.090988,-0.587207,-0.201435,-0.082970,-0.045304,-0.024865,4.674323,45,94,0.000459
1,161280,-0.547158,1.851470,-3.142172,-0.434762,1.905043,-0.955582,0.762074,0.533940,-1.190402,...,-0.277606,-0.287396,-0.269441,0.529420,-0.267365,-0.025603,0.565314,44,93,0.001419
2,151942,-0.326046,-0.217706,0.529752,-0.210276,0.381230,0.845321,-0.012136,0.122161,-1.551175,...,0.059524,-0.986212,-0.561194,0.203248,0.084215,0.197627,4.143135,42,87,0.001874
3,78663,-1.192746,0.589859,1.437212,-1.465846,0.904997,0.961699,0.720329,-0.107361,0.201824,...,-0.502046,-1.224348,0.502369,1.053619,-0.751317,-0.139799,3.044522,21,45,0.000340
4,116781,1.348242,-1.475166,-1.262404,0.105946,0.281289,1.895419,-0.398966,0.538657,0.897916,...,0.261410,-0.963344,-0.880669,0.197854,-0.058803,-0.019419,5.669640,32,67,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71197,66038,-0.882345,1.234006,1.835280,-0.543689,0.421677,-0.947680,1.170509,-0.349857,-0.392525,...,-0.234014,0.290883,0.275506,-0.012386,-0.058481,-0.085526,2.301585,18,38,0.001412
71198,125077,2.048996,-0.366883,-2.543852,-0.728626,2.387568,3.318339,-0.477944,0.791325,0.403268,...,0.380947,0.698226,-0.265129,0.219179,-0.052301,-0.066783,1.026042,34,72,0.001300
71199,154041,2.219642,-0.691482,-1.586185,-0.597533,-0.308073,-0.761185,-0.362941,-0.233413,-0.437414,...,0.139674,0.469916,-0.038320,0.775403,-0.078470,-0.063665,2.302585,42,89,0.002353
71200,40573,1.236821,0.055497,0.617927,0.882157,-0.590356,-0.588493,-0.109363,-0.124856,0.554266,...,-0.088960,0.465463,0.603302,0.437325,-0.012067,0.010836,0.693147,11,23,0.005226


In [53]:
# =============================
# 6. 최종 result_df 예측 및 저장
# =============================

# SMOTE 앙상블 예측 (확률)
smote_probs_result = smote_ensemble.predict_proba(result_df)[:, 1]

# Threshold 적용
smote_probs_result = (smote_probs_result >= 0.65).astype(int)

In [54]:

result = pd.read_csv('Result.csv')
# 결과 저장
result['Class'] = smote_probs_result

In [56]:
result['Class'].value_counts()

Class
0    71093
1      109
Name: count, dtype: int64

In [55]:
result.to_csv('classify_prediction_0414_last.csv', index=False)

print("최종 예측 결과가 'final_result.csv' 로 저장되었습니다!")


최종 예측 결과가 'final_result.csv' 로 저장되었습니다!
